# APIとは
APIは、**Application Programming Interface**の略で、ソフトウェアやプログラム、Webサービスの間をつなぐインターフェイスのこと。APIを利用することで、様々な既成のサービスを利用したり、それらと連携したりすることが出来る。

## NewsAPI演習
今回は、日本語の記事でウクライナに関する記事を収集し、pprintで出力してみる。

In [1]:
!pip install -q newsapi-python
from newsapi import NewsApiClient
from pprint import pprint

api = NewsApiClient(api_key='18e8f132860b4fdb93f267e4aa9e1e2c')

top_headlines = api.get_top_headlines(q='ウクライナ', language=None, country='jp')

pprint(top_headlines)

{'articles': [{'author': '日本経済新聞社',
               'content': None,
               'description': '【ワシントン=中村亮】ウクライナ軍が4月中旬にロシア黒海艦隊旗艦の巡洋艦「モスクワ」をミサイル攻撃で撃沈した際、米国から得た情報を活用していたことが5日、分かった。複数の米メディアが報じた。モスクワはロシア海軍の強さの象徴で、米国の関与にロシアが反発を強める可能性がある。米国防総省高官は日本経済新聞の取材で「ウクライナ軍が黒海にいるロシア船の脅威を把握したり、海からの潜在的な攻撃に対する防衛の',
               'publishedAt': '2022-05-06T08:50:03Z',
               'source': {'id': None, 'name': 'Nikkei.com'},
               'title': '米、旗艦「モスクワ」撃沈支援か ウクライナに情報提供（写真=ロイター） - 日本経済新聞',
               'url': 'https://www.nikkei.com/article/DGXZQOGN0604Y0W2A500C2000000/',
               'urlToImage': 'https://article-image-ix.nikkei.com/https%3A%2F%2Fimgix-proxy.n8s.jp%2FDSXZQO1894806006052022000000-1.jpg?ixlib=js-2.3.2&auto=format%2Ccompress&ch=Width%2CDPR&q=45&fit=crop&bg=FFFFFF&w=1200&h=630&fp-x=0.5&fp-y=0.5&fp-z=1&crop=focalpoint&s=ad831ca9815bdf596f737c1e6c3eb4ee'},
              {'author': None,
               'content': 'Your browser isnt supported anymore. Update it to '
            

# Webスクレイピングとは
Webスクレイピングとは、Webサイトから任意の情報を抽出、整形、解析する技術のこと。

## Webスクレイピング演習
今回は、SUUMOのサイトから物件情報をスクレイピングし、結果をcsvファイルに出力する。

In [13]:
!pip install -q pandas
from time import sleep
import re

from bs4 import BeautifulSoup
import requests
import pandas as pd

url = 'https://suumo.jp/chintai/tokyo/sc_shinjuku/?page{}'

d_list = []

for i in range(1, 3):
    print('d_listの大きさ：', len(d_list))
    target_url = url.format(i)
    
    print(target_url)
    
    r = requests.get(target_url)
    
    sleep(1)
    
    soup = BeautifulSoup(r.text)
    
    contents = soup.find_all('div', class_='cassetteitem')
    
    for content in contents:
        
        detail = content.find('div', class_='cassetteitem_content')
        table = content.find('table', class_='cassetteitem_other')
        
        title = detail.find('div', class_='cassetteitem_content-title').text
        address = detail.find('li', class_='cassetteitem_detail-col1').text
        access = detail.find('li', class_='cassetteitem_detail-col2').text
        age = detail.find('li', class_='cassetteitem_detail-col3').text
        
        tr_tags = table.find_all('tr', class_='js-cassette_link')
        
        for tr_tag in tr_tags:
            
            floor, price, first_fee, capacity = tr_tag.find_all('td')[2:6]
            
            fee, management_fee = price.find_all('li')
            deposit, gratuity = first_fee.find_all('li')
            madori, menseki = capacity.find_all('li')
            
            d = {
                'title': title,
                'address': address,
                'access': access,
                'age': age,
                'floor': floor.text,
                'fee': fee.text,
                'management_fee': management_fee.text,
                'deposit': deposit.text,
                'gratuity': gratuity.text,
                'madori': madori.text,
                'menseki': menseki.text
            }
            
            d_list.append(d)

df = pd.DataFrame(d_list)

df2 = df.applymap(lambda x: re.sub('\n', ' ', x))

df2.to_csv('test.csv', index=None, encoding='utf-8-sig')
print('finished to convert to csv')

d_listの大きさ： 0
https://suumo.jp/chintai/tokyo/sc_shinjuku/?page1
d_listの大きさ： 181
https://suumo.jp/chintai/tokyo/sc_shinjuku/?page2
finished to convert to csv
